In [1]:
import sys
sys.path.append("../")
import grab_test as gt
import rtde_control
import rtde_receive
from Motor_code import Motors
import UR5_Interface as ur5
import time


ModuleNotFoundError: No module named 'open_grasp'

In [5]:
from Motor_code import Motors
servoPort = "/dev/ttyACM0"
gripperController = Motors(servoPort)
gripperController.torquelimit(600) # used to be 600
gripperController.speedlimit(100)
m1 = gripperController.Motor1
m2 = gripperController.Motor2
print(m2.get_torque_enable())


Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
1


In [140]:
gripperController.torquelimit(600) # used to be 600
gripperController.openGripper()
m1.get_load()

Position of dxl ID: 1 set to 303 
Position of dxl ID: 2 set to 729 


50

In [254]:
# strategy 0: close gripper
gripperController.torquelimit(600) # used to be 600
width = 20
dTheta = gripperController.distance2theta(width)
gripperController.position(dTheta)


error: (5, 'Input/output error')

In [248]:
gripperController.torquelimit(600) # used to be 600
gripperController.openGripper()
m1.get_load()

Position of dxl ID: 1 set to 303 
Position of dxl ID: 2 set to 729 


1624

In [13]:
# strategy 1: close 1 finger until contact, then make rigid and close 2nd finger

# error margin of 10 pos units allowed
m1.set_ccw_compliance_margin(10)
# m1.set_ccw_compliance_slope(6)
m1.set_cw_compliance_margin(10)
# m1.set_cw_compliance_slope(6)


# bring m1 to near object
# m1 reset
m1.set_torque_limit(100)
m1.set_goal_position(380)
print(m1.get_present_position()) # validate margin
time.sleep(0.8)

# set torque to be 5% (this is enough to move downward, not enough torque to move up)
m1.set_torque_limit(75)
pos = m1.get_present_position()
for i in range(0, 200):
    m1.set_goal_position(pos + i)
    # print(m1.get_present_position())
    load = m1.get_load()
    abs_load = load
    if load >= 1023:
        abs_load = load - 1023
    print(f"pos: {pos + i } load: {abs_load}")
    if abs_load > 45: #corresponds to >0.8% load
        break

time.sleep(0.1)

# set current pos to new goal (to avoid extraneous movement)
m1.set_goal_position(m1.get_present_position())
time.sleep(0.4)


# make m1 rigid
# m1 config
m1.set_ccw_compliance_margin(0)
# m1.set_ccw_compliance_slope(6)
m1.set_cw_compliance_margin(0)
# m1.set_cw_compliance_slope(6)
m1.set_torque_limit(500)
time.sleep(0.3)

print(m2.get_present_position())

# close grasp
m2.set_goal_position(500)

Position of dxl ID: 1 set to 380 
303
Position of dxl ID: 1 set to 358 
pos: 358 load: 37
Position of dxl ID: 1 set to 359 
pos: 359 load: 37
Position of dxl ID: 1 set to 360 
pos: 360 load: 37
Position of dxl ID: 1 set to 361 
pos: 361 load: 37
Position of dxl ID: 1 set to 362 
pos: 362 load: 37
Position of dxl ID: 1 set to 363 
pos: 363 load: 37
Position of dxl ID: 1 set to 364 
pos: 364 load: 37
Position of dxl ID: 1 set to 365 
pos: 365 load: 37
Position of dxl ID: 1 set to 366 
pos: 366 load: 37
Position of dxl ID: 1 set to 367 
pos: 367 load: 37
Position of dxl ID: 1 set to 368 
pos: 368 load: 37
Position of dxl ID: 1 set to 369 
pos: 369 load: 37
Position of dxl ID: 1 set to 370 
pos: 370 load: 37
Position of dxl ID: 1 set to 371 
pos: 371 load: 37
Position of dxl ID: 1 set to 372 
pos: 372 load: 37
Position of dxl ID: 1 set to 373 
pos: 373 load: 37
Position of dxl ID: 1 set to 374 
pos: 374 load: 37
Position of dxl ID: 1 set to 375 
pos: 375 load: 37
Position of dxl ID: 1 set 

In [14]:
gripperController.torquelimit(600) # used to be 600
gripperController.openGripper()

Position of dxl ID: 1 set to 303 
Position of dxl ID: 2 set to 729 


In [17]:
gripperController.torquelimit(25) # used to be 600
gripperController.openGripper()

Position of dxl ID: 1 set to 303 
Position of dxl ID: 2 set to 729 


In [3]:
# strategy 2: close both fingers until contact, then rigidify and close

# error margin of 10 pos units allowed
m1.set_ccw_compliance_margin(10)
# m1.set_ccw_compliance_slope(6)
m1.set_cw_compliance_margin(10)
# m1.set_cw_compliance_slope(6)

m2.set_ccw_compliance_margin(10)
m2.set_cw_compliance_margin(10)
gripperController.torquelimit(100)

# bring gripper to near object
gripperController.position(50) # 45cm
time.sleep(0.8)

# set torque to be 5% (this is enough to move downward, not enough torque to move up)
m1.set_torque_limit(75)
m2.set_torque_limit(75)
pos1 = m1.get_present_position()
pos2 = m2.get_present_position()
m1_stop, m2_stop = False, False

load_thresh = 65

for i in range(0, 200):
    if not m1_stop:
        m1.set_goal_position(pos1 + i)
    if not m2_stop:
        m2.set_goal_position(pos2 - i)
    time.sleep(0.01)
    # print(m1.get_present_position())
    load1 = m1.get_load()
    load2 = m2.get_load()
    aload1 = load1
    aload2 = load2
    if load1 >= 1023:
        aload1 = load1 - 1023
    if load2 >= 1023:
        aload2 = load2 - 1023
    # print(f"pos1: {pos1 + i } load: {aload1}")
    # print(f"pos2: {pos2 + i } load: {aload2}")
    if aload1 > load_thresh and not m1_stop: #corresponds to >0.8% load
        m1.set_goal_position(pos1 + i)
        print(f"stopping m1 at {pos1 + i} with load {aload1}")
        m1_stop = True
    if aload2 > load_thresh and not m2_stop: #corresponds to >0.8% load
        m2.set_goal_position(pos2 - i)
        print(f"stopping m2 at {pos2 - i} with load {aload2}")
        m2_stop = True

# make gripper
print("setting gripper to high torque")
m1.set_ccw_compliance_margin(0)
m1.set_cw_compliance_margin(0)
m2.set_ccw_compliance_margin(0)
m2.set_cw_compliance_margin(0)
m1.set_torque_limit(500)
m2.set_torque_limit(500)
time.sleep(0.1)

# close grasp
print ("closing gripper")
width = 30
dTheta = gripperController.distance2theta(width)


Position of dxl ID: 1 set to 341 
Position of dxl ID: 2 set to 699 
Position of dxl ID: 1 set to 329 
Position of dxl ID: 2 set to 699 
Position of dxl ID: 1 set to 330 
Position of dxl ID: 2 set to 698 
Position of dxl ID: 1 set to 331 
Position of dxl ID: 2 set to 697 
Position of dxl ID: 1 set to 332 
Position of dxl ID: 2 set to 696 
Position of dxl ID: 1 set to 333 
Position of dxl ID: 2 set to 695 
Position of dxl ID: 1 set to 334 
Position of dxl ID: 2 set to 694 
Position of dxl ID: 1 set to 335 
Position of dxl ID: 2 set to 693 
Position of dxl ID: 1 set to 336 
Position of dxl ID: 2 set to 692 
Position of dxl ID: 1 set to 337 
Position of dxl ID: 2 set to 691 
Position of dxl ID: 1 set to 338 
Position of dxl ID: 2 set to 690 
Position of dxl ID: 1 set to 339 
Position of dxl ID: 2 set to 689 
Position of dxl ID: 1 set to 340 
Position of dxl ID: 2 set to 688 
Position of dxl ID: 1 set to 341 
Position of dxl ID: 2 set to 687 
Position of dxl ID: 1 set to 342 
Position of dx

In [6]:
gripperController.openGripper()

Position of dxl ID: 1 set to 303 
[RxPacketError] Overload error!
[RxPacketError] Overload error!
Position of dxl ID: 2 set to 729 


In [ ]:
# Full test
sleepRate = 0.75

try:
    robotIP = "192.168.0.6"
    con = rtde_control.RTDEControlInterface(robotIP)
    rec = rtde_receive.RTDEReceiveInterface(robotIP)
    servoPort = "/dev/ttyACM0"
    gripperController = Motors(servoPort)
    gripperController.torquelimit(600) # used to be 600
    gripperController.speedlimit(100)
    ur = ur5.UR5_Interface()
    ur.gripperController = gripperController
    try:
        ur.c = con
        ur.r = rec
        ur.gripperController = gripperController
    except Exception as e:
        raise(e)
    else:
        print("UR5 + Gripper Interface Established")
            
    # print(f"res: {projectToWorldCoords(res)} ")
    # ur.openGripper() # Open gripper
    # ur.testRoutine()
    homePose = ur.getPose()
    x_mod = 0.0
    y_mod = 0.0
    z_mod = 0.0
    gt.moveToBlock(grasp_pose, ur) 
    print("Done moving to block")
    ur.closeGripper(9) 
    time.sleep(sleepRate)
    gt.moveBackFromBlock(homePose, ur)
    ur.openGripper()
    gripperController.openGripper()
    gripperController.disconnect()
    ur.c.disconnect()
    ur.r.disconnect()
except Exception as e:
    gripperController.openGripper()
    gripperController.disconnect()
    ur.c.disconnect()
    ur.r.disconnect()
    raise(e)